<a href="https://colab.research.google.com/github/Shaurya016/rag-project/blob/main/MainFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install pandas sentence-transformers faiss-cpu transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [6]:
import pandas as pd
from google.colab import files

uploaded = files.upload()  # Upload "Training Dataset.csv" from your computer
df = pd.read_csv("Training Dataset.csv")
df.dropna(inplace=True)
df.head()


Saving Training Dataset.csv to Training Dataset.csv


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
5,LP001011,Male,Yes,2,Graduate,Yes,5417,4196.0,267.0,360.0,1.0,Urban,Y


In [7]:
def row_to_text(row):
    return f"Applicant ID {row['Loan_ID']} is a {row['Gender']} who is {row['Married']} and is {row['Education']}. " \
           f"The applicant is {'self-employed' if row['Self_Employed'] == 'Yes' else 'not self-employed'}, " \
           f"has an income of {row['ApplicantIncome']}, requested a loan amount of {row['LoanAmount']} and " \
           f"their loan was {'approved' if row['Loan_Status'] == 'Y' else 'not approved'}."

texts = df.apply(row_to_text, axis=1).tolist()


In [8]:
from sentence_transformers import SentenceTransformer

embed_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embed_model.encode(texts, show_progress_bar=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

In [9]:
import faiss
import numpy as np

dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Retrieval function
def get_top_k_chunks(query, k=3):
    query_embedding = embed_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [texts[i] for i in indices[0]]


In [10]:
from transformers import pipeline

# Flan-T5 is a good small model for text generation
generator = pipeline("text2text-generation", model="google/flan-t5-base")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [11]:
def ask_rag_bot(query):
    context = " ".join(get_top_k_chunks(query))
    prompt = f"Use the following data to answer:\n{context}\n\nQuestion: {query}"
    response = generator(prompt, max_length=150, do_sample=True)
    return response[0]['generated_text']

# Example
ask_rag_bot("How many people were self-employed and got their loans approved?")


Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'3'

In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import io

# Sample questions
sample_questions = [
    "How does education level affect loan approval?",
    "What is the income range of approved loan applicants?",
    "Are self-employed people more likely to get loans approved?",
    "How many female applicants got their loans approved?",
    "What loan amount is typically approved?"
]

# Chat history list
chat_history = []

# Widgets
input_box = widgets.Text(
    value='',
    placeholder='Ask a question...',
    description='Query:',
    layout=widgets.Layout(width='80%')
)

output_box = widgets.Output()

dropdown = widgets.Dropdown(
    options=['Select a sample question'] + sample_questions,
    description='Samples:',
    layout=widgets.Layout(width='80%')
)

download_button = widgets.Button(
    description='📥 Download Chat History',
    button_style='success'
)

# Retrieval function
def get_top_k_chunks(query, k=3):
    query_embedding = embed_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return [texts[i] for i in indices[0]]

def ask_rag_bot(query):
    context = " ".join(get_top_k_chunks(query))
    prompt = f"Use the following data to answer:\n{context}\n\nQuestion: {query}"
    response = generator(prompt, max_length=150, do_sample=True)
    return response[0]['generated_text']

def ask_and_display(query):
    if not query.strip():
        return
    if query.lower() == 'exit':
        with output_box:
            clear_output()
            print("Chat ended.")
        return

    answer = ask_rag_bot(query)
    timestamp = datetime.now().strftime("%H:%M:%S")
    chat_history.append(f"[{timestamp}] Q: {query}\nA: {answer}\n")

    with output_box:
        clear_output()
        for line in chat_history:
            print(line.strip())

    input_box.value = ''
    dropdown.value = 'Select a sample question'

# Handlers
def on_text_submit(sender):
    ask_and_display(sender.value)

def on_dropdown_change(change):
    if change['new'] != 'Select a sample question':
        input_box.value = change['new']
        ask_and_display(change['new'])


def on_download_click(b):
    file_content = "\n".join(chat_history)
    buffer = io.StringIO(file_content)
    buffer.seek(0)

    # Write to file before downloading
    with open("chat_history.txt", "w") as f:
        f.write(file_content)

    from google.colab import files
    files.download('chat_history.txt')


# Bind events
input_box.on_submit(on_text_submit)
dropdown.observe(on_dropdown_change, names='value')
download_button.on_click(on_download_click)

# Display interface
display(dropdown, input_box, download_button, output_box)

Dropdown(description='Samples:', layout=Layout(width='80%'), options=('Select a sample question', 'How does ed…

Text(value='', description='Query:', layout=Layout(width='80%'), placeholder='Ask a question...')

Button(button_style='success', description='📥 Download Chat History', style=ButtonStyle())

Output()

Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>